# Homenet

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Homenet").config("spark.memory.offHeap.enabled", "true").config(
    "spark.memory.offHeap.size", "10g").getOrCreate()

In [ ]:
adresser = spark.read.json("spark-warehouse/adresser.json", multiLine=True)
adresser.createOrReplaceTempView("adresser")

maklare = spark.read.json("spark-warehouse/maklare.json", multiLine=True)
maklare.createOrReplaceTempView("maklare")

objekt = spark.read.json("spark-warehouse/objekt.json", multiLine=True)
objekt.createOrReplaceTempView("objekt")

personer = spark.read.json("spark-warehouse/personer.json", multiLine=True)
personer.createOrReplaceTempView("personer")

In [ ]:
objekt.printSchema()

In [ ]:
objekt.show(1,0)

In [ ]:
result = spark.sql("SELECT * FROM personer LIMIT 5")
result.show()

In [ ]:
result = spark.sql("""
    SELECT * FROM objekt, personer 
        WHERE objekt.saljare = personer.id 
        LIMIT 50
""")
result.show()

C. Vilken storlek på lägenheter säljer snabbast i Skåne? De som är under 50kvm, mellan 50-100kvm eller över 100kvm?

In [ ]:
result = spark.sql("""
SELECT "Mindre än 50 kvm" AS storlek, ceil(avg(datediff(avslutat, saljstart))) AS dagar FROM objekt WHERE avslutat IS NOT NULL 
AND storlek < 50
UNION
SELECT "Mellan 50–100 kvm" AS storlek, ceil(avg(datediff(avslutat, saljstart))) AS dagar FROM objekt WHERE avslutat IS NOT NULL 
AND storlek BETWEEN 50 AND 100
UNION
SELECT "Större än 50 kvm" AS storlek, ceil(avg(datediff(avslutat, saljstart))) AS dagar FROM objekt WHERE avslutat IS NOT NULL 
AND storlek > 100
ORDER BY dagar LIMIT 1
""") 
result.show()